In [1]:
import numpy as np 
import pandas as pd
import re


In [7]:
df_nlp = pd.read_csv("df_file.csv")

In [8]:
df_nlp.info()
df_nlp.drop_duplicates(inplace=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2225 entries, 0 to 2224
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Text    2225 non-null   object
 1   Label   2225 non-null   int64 
dtypes: int64(1), object(1)
memory usage: 34.9+ KB


In [4]:
# df_nlp.duplicated().sum()

In [ ]:
df_nlp["Text"] = df_nlp["Text"].str.lower() #Mengubah semua kata menjadi huruf kecil

import re

def remove_html_tags(text): # Fungsi untuk menghapus tag HTML dari teks.

    html_tag_pattern = re.compile("<.*?>")  
    cleaned_text = html_tag_pattern.sub(r"", text)  
    return cleaned_text

df_nlp["Text"] = df_nlp["Text"].apply(remove_html_tags)


In [ ]:
def remove_url(text):
    pattern = re.compile(r"https?://\S+|www\S+")
    return pattern.sub(r"", text)
df_nlp["Text"] = df_nlp["Text"].apply(remove_url)



In [5]:
import string, time
punc = string.punctuation
def remove_punctuations(text):
    return text.translate(str.maketrans("","",punc))
df_nlp["Text"] = df_nlp["Text"].apply(remove_punctuations)
df_nlp["Text"][0]

'budget to set scene for election\n \n gordon brown will seek to put the economy at the centre of labours bid for a third term in power when he delivers his ninth budget at 1230 gmt he is expected to stress the importance of continued economic stability with low unemployment and interest rates the chancellor is expected to freeze petrol duty and raise the stamp duty threshold from â£60000 but the conservatives and lib dems insist voters face higher taxes and more meanstesting under labour\n \n treasury officials have said there will not be a preelection giveaway but mr brown is thought to have about â£2bn to spare\n \n  increase in the stamp duty threshold from â£60000 \n   a freeze on petrol duty \n   an extension of tax credit scheme for poorer families \n   possible help for pensioners the stamp duty threshold rise is intended to help first time buyers  a likely theme of all three of the main parties general election manifestos ten years ago buyers had a much greater chance of avoid

In [6]:
label_mapping = {0: 'Politics', 1: 'Sport', 2: 'Technology', 3: 'Entertainment', 4: 'Business'}

df_nlp['Label'] = df_nlp['Label'].map(label_mapping)

print(df_nlp)

# # mengubah label 1, 2 ,3 ,4 ke kategorikal terlebih dahulu

                                                   Text     Label
0     budget to set scene for election\n \n gordon b...  Politics
1     army chiefs in regiments decision\n \n militar...  Politics
2     howard denies split over id cards\n \n michael...  Politics
3     observers to monitor uk election\n \n minister...  Politics
4     kilroy names election seat target\n \n exchat ...  Politics
...                                                 ...       ...
2220  india opens skies to competition\n \n india wi...  Business
2221  yukos bankruptcy not us matter\n \n russian au...  Business
2222  survey confirms property slowdown\n \n governm...  Business
2223  high fuel prices hit bas profits\n \n british ...  Business
2224  us trade gap hits record in 2004\n \n the gap ...  Business

[2127 rows x 2 columns]


In [7]:
df_nlp

,Text,Label
0,budget to set scene for election\n \n gordon b...,Politics
1,army chiefs in regiments decision\n \n militar...,Politics
2,howard denies split over id cards\n \n michael...,Politics
3,observers to monitor uk election\n \n minister...,Politics
4,kilroy names election seat target\n \n exchat ...,Politics
...,...,...
2220,india opens skies to competition\n \n india wi...,Business
2221,yukos bankruptcy not us matter\n \n russian au...,Business
2222,survey confirms property slowdown\n \n governm...,Business
2223,high fuel prices hit bas profits\n \n british ...,Business


In [8]:
df_encoded = pd.get_dummies(df_nlp, columns=['Label'], prefix='', prefix_sep='')
df_encoded.replace({True:1, False:0})

,Text,Business,Entertainment,Politics,Sport,Technology
0,budget to set scene for election\n \n gordon b...,0,0,1,0,0
1,army chiefs in regiments decision\n \n militar...,0,0,1,0,0
2,howard denies split over id cards\n \n michael...,0,0,1,0,0
3,observers to monitor uk election\n \n minister...,0,0,1,0,0
4,kilroy names election seat target\n \n exchat ...,0,0,1,0,0
...,...,...,...,...,...,...
2220,india opens skies to competition\n \n india wi...,1,0,0,0,0
2221,yukos bankruptcy not us matter\n \n russian au...,1,0,0,0,0
2222,survey confirms property slowdown\n \n governm...,1,0,0,0,0
2223,high fuel prices hit bas profits\n \n british ...,1,0,0,0,0


In [9]:
text = df_encoded['Text'].values
label = df_encoded[['Business', 'Entertainment', 'Politics', 'Sport', 'Technology']].values


In [10]:
from sklearn.model_selection import train_test_split
text_latih, text_test, label_latih, label_test = train_test_split(text, label, test_size=0.2)

In [11]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences


In [3]:
class myCallback(tf.keras.callbacks.Callback):
  def on_epoch_end(self, epoch, logs={}):
    if(logs.get('accuracy') and logs.get('val_accuracy')>0.9):
      print("\nAkurasi telah mencapai >90%!")
      self.model.stop_training = True
callbacks = myCallback()

In [12]:
tokenizer = Tokenizer(num_words=10000)
tokenizer.fit_on_texts(text_latih) 

 
sekuens_latih = tokenizer.texts_to_sequences(text_latih)
sekuens_test = tokenizer.texts_to_sequences(text_test)
 
padded_latih = pad_sequences(sekuens_latih) 
padded_test = pad_sequences(sekuens_test)

In [1]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras import layers

model = Sequential()
model.add(layers.Embedding(input_dim=10000, output_dim=500))
model.add(layers.LSTM(128, return_sequences=True, recurrent_dropout=0.2))
model.add(layers.LSTM(64, recurrent_dropout=0.2))
model.add(layers.BatchNormalization())
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dropout(0.3))
model.add(layers.Dense(128, activation='relu'))
model.add(layers.Dropout(0.3))
model.add(layers.Dense(5, activation='softmax'))

optimizer = tf.keras.optimizers.Adam(learning_rate=0.0005)


model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])


In [19]:
# Pastikan TensorFlow menggunakan GPU
physical_devices = tf.config.list_physical_devices('GPU')
if len(physical_devices) > 0:
    tf.config.experimental.set_memory_growth(physical_devices[0], True)


In [2]:
num_epochs = 32
history = model.fit(padded_latih, label_latih, epochs=num_epochs,
                    validation_data=(padded_test, label_test), verbose=2, callbacks=callbacks)

NameError: name 'padded_latih' is not defined